In [59]:
# default_exp sklearn_lda

# Preprocessing

In [60]:
#export
import pandas as pd

In [61]:
#export
import jieba

In [63]:
#export
def make_df(csv_name):
    '''Use jieba, create data frame.'''
    df = pd.read_csv(csv_name)
    df = df.dropna(subset=['Content'])
    df['text'] = df.Content.apply(lambda x: " ".join(jieba.cut(x)))
    return df

In [64]:
affirmative = make_df("affirmative.csv")

In [65]:
affirmative.head()

,Group,Students,Content,text
0,第1组,正三,慕课将分布于世界各地的最优质的教育资源聚集到一起，让任何有学习愿望的人能够低成本的，通常是免...,慕课 将 分布 于 世界各地 的 最 优质 的 教育资源 聚集 到 一起 ， 让 任何 有 ...
1,第1组,正二,"在慕课发展过程中的现阶段，中国最大的慕课平台icourse163的用户人数突破100万,与其...",在 慕课 发展 过程 中 的 现阶段 ， 中国 最大 的 慕课 平台 icourse163 ...
2,第1组,正一,"研究发现,在慕课融入的课堂学习中,学习者情感体验丰富,知识技能以及元认知能力得到提升,思想观...","研究 发现 , 在 慕课 融入 的 课堂 学习 中 , 学习者 情感 体验 丰富 , 知识 ..."
3,第1组,正三,慕课在保证教育质量的同时，降低提供教育的成本，给社会带来的憧憬。任何人任何时候再任何地方，都...,慕课 在 保证 教育 质量 的 同时 ， 降低 提供 教育 的 成本 ， 给 社会 带来 的...
4,第1组,正一,对方反一辩友也说是可能出现的欢快气氛，传统课堂集体聆听教师单方面赐予的知识，这难道不是一种容...,对方 反一 辩友 也 说 是 可能 出现 的 欢快 气氛 ， 传统 课堂 集体 聆听 教师 ...


In [66]:
negative = make_df("negative.csv")

In [67]:
negative.head()

,Group,Students,Content,text
0,第1组,反一,通俗地说，慕课是大规模的网络开放课程。1.传统课堂中可能出现的欢快的气氛，慕课没有；2.传统...,通俗 地说 ， 慕课 是 大规模 的 网络 开放 课程 。 1 . 传统 课堂 中 可能 出...
1,第1组,反一,通俗地说，慕课是大规模的网络开放课程。我方观点为，慕课不能代替传统课堂1.传统课堂中可能出现...,通俗 地说 ， 慕课 是 大规模 的 网络 开放 课程 。 我方 观点 为 ， 慕课 不能 ...
2,第1组,反一,"正一观点中有这样一句话“相比在线交流,学习者偏好面对面的交流讨论”这不正是传统课堂所能给予的吗","正一 观点 中有 这样 一句 话 “ 相比 在线 交流 , 学习者 偏好 面对面 的 交流 ..."
3,第1组,反一,正二给出的数字中，说明不了慕课的替代性，毕竟中国学生庞大的基数在这。,正二 给出 的 数字 中 ， 说明 不了 慕课 的 替代性 ， 毕竟 中国 学生 庞大 的 ...
4,第1组,反一,正一说的独立思考问题，传统课堂的弹性较大，课堂设计掌握在老师学生手中，可以做到因材施教，针对...,正一说 的 独立思考 问题 ， 传统 课堂 的 弹性 较大 ， 课堂 设计 掌握 在 老师 ...


# LDA

In [68]:
#export
import pyLDAvis 
import pyLDAvis.sklearn 

In [76]:
#export
pyLDAvis.enable_notebook()

In [70]:
#export
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
from sklearn.decomposition import LatentDirichletAllocation

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [71]:
#export
def get_custom_stopwords(stop_words_file, encoding = 'utf-8'):
    with open(stop_words_file, encoding = encoding) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [72]:
stopwords = get_custom_stopwords("stopwords.txt", encoding='utf-8') # HIT停用词词典
max_df = 0.9 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 5 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
n_features = 1000 # 最大提取特征数量
n_top_words = 20 # 显示主题下关键词的时候，显示多少个
col_content = "text" # 说明其中的文本信息所在列名称

In [73]:
#export
def lda_on_chinese_articles_with_param(df, n_topics, 
                            col_content, 
                            stopwords, 
                            n_features, 
                            max_df, 
                            min_df,
                            n_top_words):
    articles_cutted = df[col_content].apply(chinese_word_cut)
    vect = CountVectorizer(max_df = max_df, 
                       min_df = min_df, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))
    tf = vect.fit_transform(articles_cutted)
    lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                    learning_method='online',
                                    learning_offset=50,
                                    random_state=0)
    lda.fit(tf)
    print_top_words(lda, vect.get_feature_names(), n_top_words)
    return lda, tf, vect

In [74]:
#export
def lda_on_chinese_articles(df, n_topics):
    return lda_on_chinese_articles_with_param(df, n_topics, 
                            col_content = col_content, 
                            stopwords = stopwords, 
                            n_features = n_features, 
                            max_df = max_df, 
                            min_df = min_df,
                            n_top_words = n_top_words)

In [75]:
lda, tf, vect = lda_on_chinese_articles(df = affirmative, n_topics = 3)
pyLDAvis.sklearn.prepare(lda, tf, vect)

D:\install\miniconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｎｇ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Topic #0:
学习 学生 交流 课程 学习者 视频 时间 网络 讨论 进行 教师 课堂 自主 知识 教学 兴趣 实现 促进 合作 高等教育
Topic #1:
教育 发展 传统 大学 已经 取代 我国 不能 社会 课堂 问题 不是 认为 时代 现在 目前 技术 国家 高等教育 高校
Topic #2:
学生 课堂 传统 学习 教师 老师 教学 进行 方式 过程 视频 知识 课程 需要 一种 不是 问题 时间 没有 模式


D:\install\miniconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.057797 -0.125530       1        1  49.785762
0     -0.158830  0.091751       2        1  25.893650
1      0.216627  0.033779       3        1  24.320588, topic_info=    Category        Freq  Term       Total  loglift  logprob
222  Default  585.000000    学习  585.000000  30.0000  30.0000
376  Default  216.000000    教育  216.000000  29.0000  29.0000
228  Default  693.000000    学生  693.000000  28.0000  28.0000
160  Default  193.000000    发展  193.000000  27.0000  27.0000
62   Default   73.000000    交流   73.000000  26.0000  26.0000
224  Default   61.000000   学习者   61.000000  25.0000  25.0000
92   Default  402.000000    传统  402.000000  24.0000  24.0000
213  Default   73.000000    大学   73.000000  23.0000  23.0000
382  Default   97.000000    方式   97.000000  22.0000  22.0000
261  Default   70.000000    已经   70.000000  21.0000  21.0000
319  Default   38.000000    我国   38.000000  20.0000  20.0000
541  Default  153.000000    老师  153.000000  19.0000  19.0000
630  Default   94.000000    过程   94.000000  18.0000  18.0000
505  Default   39.000000    社会   39.000000  17.0000  17.0000
589  Default   54.000000    讨论   54.000000  16.0000  16.0000
374  Default  195.000000    教师  195.000000  15.0000  15.0000
581  Default  141.000000    视频  141.000000  14.0000  14.0000
389  Default  109.000000    时间  109.000000  13.0000  13.0000
538  Default   66.000000    网络   66.000000  12.0000  12.0000
25   Default   61.000000    不能   61.000000  11.0000  11.0000
387  Default   31.000000    时代   31.000000  10.0000  10.0000
6    Default   72.000000    一种   72.000000   9.0000   9.0000
192  Default   28.000000    国家   28.000000   8.0000   8.0000
123  Default   31.000000    兴趣   31.000000   7.0000   7.0000
479  Default   33.000000    目前   33.000000   6.0000   6.0000
550  Default   47.000000    自主   47.000000   5.0000   5.0000
465  Default   38.000000    现在   38.000000   4.0000   4.0000
165  Default   87.000000    取代   87.000000   3.0000   3.0000
176  Default   27.000000    合作   27.000000   2.0000   2.0000
605  Default  417.000000    课堂  417.000000   1.0000   1.0000
..       ...         ...   ...         ...      ...      ...
660   Topic3   19.619186    阶段   21.521504   1.3213  -5.1793
273   Topic3   14.574011    建设   15.947015   1.3238  -5.4766
87    Topic3   17.491873    优秀   19.258952   1.3176  -5.2941
479   Topic3   29.353645    目前   33.023484   1.2960  -4.7764
160   Topic3  162.778108    发展  193.569349   1.2406  -3.0635
379   Topic3   14.036819    文静   15.867362   1.2913  -5.5142
261   Topic3   52.365656    已经   70.217550   1.1205  -4.1976
213   Topic3   52.467481    大学   73.383873   1.0783  -4.1957
465   Topic3   30.018153    现在   38.785302   1.1576  -4.7540
134   Topic3   18.982336    出现   23.200291   1.2132  -5.2123
30    Topic3   16.269575    世界   19.341038   1.2409  -5.3665
25    Topic3   37.832990    不能   61.320229   0.9309  -4.5227
681   Topic3   26.007129    高校   38.961303   1.0096  -4.8975
450   Topic3   15.217937    满足   18.517383   1.2176  -5.4334
165   Topic3   41.323477    取代   87.364940   0.6652  -4.4344
92    Topic3   96.792568    传统  402.618552  -0.0116  -3.5833
585   Topic3   32.349961    认为   69.190288   0.6536  -4.6792
328   Topic3   28.786277    技术   57.259845   0.7261  -4.7959
267   Topic3   22.634469    平台   39.325424   0.8614  -5.0364
659   Topic3   35.984792    问题  106.755696   0.3264  -4.5727
23    Topic3   34.629066    不是  113.972096   0.2226  -4.6112
368   Topic3   23.711259  教学模式   47.537040   0.7183  -4.9899
682   Topic3   26.162763  高等教育   60.021366   0.5835  -4.8915
2     Topic3   23.867739    一定   55.105412   0.5771  -4.9833
605   Topic3   37.443476    课堂  417.495616  -0.9976  -4.5330
428   Topic3   25.236943    模式   87.270561   0.1731  -4.9275
439   Topic3   25.414306    没有  106.968073  -0.0234  -4.9205
89    Topic3   20.981978    优质   41.827379   0.7240  -5.1122
608   Topic3   24.783549   

```python
TypeError: __init__() got an unexpected keyword argument 'n_topics'
```

一般出现这种问题都是程序中字母写错、漏写之类的问题

————————————————

版权声明：本文为CSDN博主「zhuimengshaonian66」的原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/zhuimengshaonian66/article/details/81700959

`n_components` 参数名称修改了。

In [50]:
lda, tf, vect = lda_on_chinese_articles(df = negative, n_topics = 3)
pyLDAvis.sklearn.prepare(lda, tf, vect)

D:\install\miniconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｎｇ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Topic #0:
课堂 学生 传统 学习 没有 取代 教师 交流 老师 不能 互动 视频 认为 知识 师生 可能 面对面 合作 进行 不是
Topic #1:
学习 课程 学生 教育 教学 学习者 教师 发展 知识 问题 需要 无法 我国 不能 存在 进行 没有 认为 平台 方式
Topic #2:
传统 课堂 学生 现在 教育 进步 发展 高校 效果 韩高英 不是 无法 取代 中国 线下 实际 讨论 大学生 网上 时间


D:\install\miniconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.137078  0.085646       1        1  55.111775
0      0.017437 -0.145236       2        1  32.684201
2     -0.154515  0.059590       3        1  12.204024, topic_info=    Category        Freq  Term       Total  loglift  logprob
535  Default  393.000000    课堂  393.000000  30.0000  30.0000
69   Default  365.000000    传统  365.000000  29.0000  29.0000
537  Default  245.000000    课程  245.000000  28.0000  28.0000
196  Default  530.000000    学生  530.000000  27.0000  27.0000
409  Default   37.000000    现在   37.000000  26.0000  26.0000
391  Default  166.000000    没有  166.000000  25.0000  25.0000
132  Default  102.000000    发展  102.000000  24.0000  24.0000
137  Default  107.000000    取代  107.000000  23.0000  23.0000
602  Default   55.000000    高校   55.000000  22.0000  22.0000
326  Default  154.000000    教育  154.000000  21.0000  21.0000
48   Default   55.000000    互动   55.000000  20.0000  20.0000
50   Default   78.000000    交流   78.000000  19.0000  19.0000
336  Default   76.000000    无法   76.000000  18.0000  18.0000
596  Default   43.000000   韩高英   43.000000  17.0000  17.0000
189  Default  382.000000    学习  382.000000  16.0000  16.0000
191  Default   95.000000   学习者   95.000000  15.0000  15.0000
564  Default   22.000000    进步   22.000000  14.0000  14.0000
313  Default   33.000000    效果   33.000000  13.0000  13.0000
223  Default   43.000000    师生   43.000000  12.0000  12.0000
480  Default   91.000000    老师   91.000000  11.0000  11.0000
475  Default   28.000000    网上   28.000000  10.0000  10.0000
17   Default   85.000000    不是   85.000000   9.0000   9.0000
208  Default   29.000000    实际   29.000000   8.0000   8.0000
28   Default   22.000000    中国   22.000000   7.0000   7.0000
264  Default   31.000000    情况   31.000000   6.0000   6.0000
518  Default   31.000000    讨论   31.000000   5.0000   5.0000
183  Default   23.000000   大学生   23.000000   4.0000   4.0000
461  Default   15.000000    线下   15.000000   3.0000   3.0000
595  Default   36.000000   面对面   36.000000   2.0000   2.0000
338  Default   60.000000    时间   60.000000   1.0000   1.0000
..       ...         ...   ...         ...      ...      ...
311   Topic3    4.275468    放在    5.303528   1.8879  -5.8414
387   Topic3   12.447864    氛围   16.134799   1.8440  -4.7727
15    Topic3    4.739040    不好    6.171447   1.8393  -5.7384
103   Topic3    4.614922    冲击    6.139037   1.8180  -5.7650
146   Topic3    5.118667    同意    6.906186   1.8039  -5.6614
29    Topic3    6.913173   中国化    9.520767   1.7834  -5.3608
51    Topic3    5.604218   交流会    7.874333   1.7633  -5.5707
28    Topic3   15.347509    中国   22.533439   1.7194  -4.5633
183   Topic3   14.662490   大学生   23.118634   1.6481  -4.6090
313   Topic3   20.397175    效果   33.663786   1.6024  -4.2788
475   Topic3   14.562421    网上   28.211891   1.4421  -4.6158
208   Topic3   14.845258    实际   29.311865   1.4231  -4.5966
596   Topic3   18.636309   韩高英   43.785640   1.2492  -4.3691
518   Topic3   14.700385    讨论   31.639051   1.3369  -4.6064
602   Topic3   20.661917    高校   55.192832   1.1209  -4.2660
69    Topic3   67.865793    传统  365.842464   0.4187  -3.0767
535   Topic3   64.894125    课堂  393.521021   0.3010  -3.1215
420   Topic3    7.578386    电脑   12.426259   1.6089  -5.2689
264   Topic3   12.455764    情况   31.037450   1.1904  -4.7721
132   Topic3   21.210795    发展  102.581642   0.5273  -4.2397
196   Topic3   45.561604    学生  530.736552  -0.3518  -3.4752
17    Topic3   18.518694    不是   85.672735   0.5717  -4.3755
326   Topic3   21.985357    教育  154.573813   0.1531  -4.2039
336   Topic3   15.874527    无法   76.759756   0.5274  -4.5295
364   Topic3   10.331037   杜秋艳   24.835933   1.2263  -4.9591
338   Topic3   14.021038    时间   60.600830   0.6397  -4.6537
137   Topic3   15.627000    取代  107.465847   0.1752  -4.5452
480   Topic3   13.134942    老师   91.787935   0.1592  -4.7190
489   Topic3   11.607482   

In [56]:
pyLDAvis.sklearn.prepare(lda, tf, vect)

D:\install\miniconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.137078  0.085646       1        1  55.111775
0      0.017437 -0.145236       2        1  32.684201
2     -0.154515  0.059590       3        1  12.204024, topic_info=    Category        Freq  Term       Total  loglift  logprob
535  Default  393.000000    课堂  393.000000  30.0000  30.0000
69   Default  365.000000    传统  365.000000  29.0000  29.0000
537  Default  245.000000    课程  245.000000  28.0000  28.0000
196  Default  530.000000    学生  530.000000  27.0000  27.0000
409  Default   37.000000    现在   37.000000  26.0000  26.0000
391  Default  166.000000    没有  166.000000  25.0000  25.0000
132  Default  102.000000    发展  102.000000  24.0000  24.0000
137  Default  107.000000    取代  107.000000  23.0000  23.0000
602  Default   55.000000    高校   55.000000  22.0000  22.0000
326  Default  154.000000    教育  154.000000  21.0000  21.0000
48   Default   55.000000    互动   55.000000  20.0000  20.0000
50   Default   78.000000    交流   78.000000  19.0000  19.0000
336  Default   76.000000    无法   76.000000  18.0000  18.0000
596  Default   43.000000   韩高英   43.000000  17.0000  17.0000
189  Default  382.000000    学习  382.000000  16.0000  16.0000
191  Default   95.000000   学习者   95.000000  15.0000  15.0000
564  Default   22.000000    进步   22.000000  14.0000  14.0000
313  Default   33.000000    效果   33.000000  13.0000  13.0000
223  Default   43.000000    师生   43.000000  12.0000  12.0000
480  Default   91.000000    老师   91.000000  11.0000  11.0000
475  Default   28.000000    网上   28.000000  10.0000  10.0000
17   Default   85.000000    不是   85.000000   9.0000   9.0000
208  Default   29.000000    实际   29.000000   8.0000   8.0000
28   Default   22.000000    中国   22.000000   7.0000   7.0000
264  Default   31.000000    情况   31.000000   6.0000   6.0000
518  Default   31.000000    讨论   31.000000   5.0000   5.0000
183  Default   23.000000   大学生   23.000000   4.0000   4.0000
461  Default   15.000000    线下   15.000000   3.0000   3.0000
595  Default   36.000000   面对面   36.000000   2.0000   2.0000
338  Default   60.000000    时间   60.000000   1.0000   1.0000
..       ...         ...   ...         ...      ...      ...
311   Topic3    4.275468    放在    5.303528   1.8879  -5.8414
387   Topic3   12.447864    氛围   16.134799   1.8440  -4.7727
15    Topic3    4.739040    不好    6.171447   1.8393  -5.7384
103   Topic3    4.614922    冲击    6.139037   1.8180  -5.7650
146   Topic3    5.118667    同意    6.906186   1.8039  -5.6614
29    Topic3    6.913173   中国化    9.520767   1.7834  -5.3608
51    Topic3    5.604218   交流会    7.874333   1.7633  -5.5707
28    Topic3   15.347509    中国   22.533439   1.7194  -4.5633
183   Topic3   14.662490   大学生   23.118634   1.6481  -4.6090
313   Topic3   20.397175    效果   33.663786   1.6024  -4.2788
475   Topic3   14.562421    网上   28.211891   1.4421  -4.6158
208   Topic3   14.845258    实际   29.311865   1.4231  -4.5966
596   Topic3   18.636309   韩高英   43.785640   1.2492  -4.3691
518   Topic3   14.700385    讨论   31.639051   1.3369  -4.6064
602   Topic3   20.661917    高校   55.192832   1.1209  -4.2660
69    Topic3   67.865793    传统  365.842464   0.4187  -3.0767
535   Topic3   64.894125    课堂  393.521021   0.3010  -3.1215
420   Topic3    7.578386    电脑   12.426259   1.6089  -5.2689
264   Topic3   12.455764    情况   31.037450   1.1904  -4.7721
132   Topic3   21.210795    发展  102.581642   0.5273  -4.2397
196   Topic3   45.561604    学生  530.736552  -0.3518  -3.4752
17    Topic3   18.518694    不是   85.672735   0.5717  -4.3755
326   Topic3   21.985357    教育  154.573813   0.1531  -4.2039
336   Topic3   15.874527    无法   76.759756   0.5274  -4.5295
364   Topic3   10.331037   杜秋艳   24.835933   1.2263  -4.9591
338   Topic3   14.021038    时间   60.600830   0.6397  -4.6537
137   Topic3   15.627000    取代  107.465847   0.1752  -4.5452
480   Topic3   13.134942    老师   91.787935   0.1592  -4.7190
489   Topic3   11.607482   

参考 https://github.com/bmabey/pyLDAvis/issues/132
    
```python
D:\install\miniconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
```

重新安装后依然没有解决。

In [55]:
pyLDAvis.__version__

'2.1.2'

In [57]:
pd.__version__

'0.24.2'

In [ ]:
# !pip install pyldavis

In [78]:
import pickle as pkl
with open("model/sklearn-lda.pkl", 'wb') as fp:
    pkl.dump(lda, fp)

In [79]:
with open("model/sklearn-lda.pkl", 'rb') as fp:
    model0 = pkl.load(fp)
    print(model0.__class__)

<class 'sklearn.decomposition.online_lda.LatentDirichletAllocation'>
